In [2]:
import random
import time, datetime
import os
from functools import reduce
import subprocess
from datetime import datetime, timedelta
from functools import reduce
import pandas as pd
import pathlib
from dfply import *
from lifetimes import BetaGeoFitter
import seaborn as sb
import matplotlib.pyplot as plt
import scipy
import numpy as np
from sklearn.metrics import mean_absolute_error

In [3]:
%matplotlib inline

## training and testing data

In [4]:
path = '/tmp/data/pengcheng/cltv-summary/'

In [5]:
test_start_date = datetime.strptime('2019-07-01', "%Y-%m-%d").date()
test_end_date = datetime.strptime('2019-12-31', "%Y-%m-%d").date()
test_duration = (test_end_date - test_start_date).days/7.0


In [6]:
training_path = os.path.join(path,'17.pkl')
testing_path = os.path.join(path,'17_testing.pkl')

### training data

In [7]:
training_df = pd.read_pickle(training_path)


In [10]:
len(training_df)

58625648

In [11]:
training_df.recency.quantile([.25, .5, .75, .85, .95, .99])

0.25     0.000000
0.50     4.142857
0.75    16.428571
0.85    21.000000
0.95    24.142857
0.99    25.285714
Name: recency, dtype: float64

In [12]:
training_df.frequency.quantile([.25, .5, .75, .85, .95, .99])

0.25     0.0
0.50     2.0
0.75     7.0
0.85    12.0
0.95    24.0
0.99    48.0
Name: frequency, dtype: float64

In [13]:
training_df >> mask (training_df.frequency > 1000) >> head(2)

,customer_id,frequency,recency,monetary_value,T
15,318796688,2952,23.714286,129.730103,23.714286
48,1010352655,2564,25.571429,118.245613,25.714286


In [14]:
training_df_filter = training_df[training_df.frequency < training_df.frequency.quantile(.99)]

In [15]:
training_df_filter.frequency.quantile([.25, .5, .75, .85, .95, .99])

0.25     0.0
0.50     2.0
0.75     7.0
0.85    11.0
0.95    22.0
0.99    36.0
Name: frequency, dtype: float64

In [16]:
training_df_filter.monetary_value.quantile([.25, .5, .75, .85, .95, .99])

0.25     35.000000
0.50     89.800003
0.75    160.666672
0.85    214.517242
0.95    399.000000
0.99    499.000000
Name: monetary_value, dtype: float64

### testing data

In [17]:
testing_df = pd.read_pickle(testing_path)


In [18]:
len(testing_df)

46885522

In [19]:
testing_df.head(5)

,customer_id,frequency,recency,monetary_value,T
0,378479488,50,21.142857,95.333336,21.285714
1,307326802,14,24.428571,52.133335,25.857143
2,9167224,4,2.571429,75.400002,9.000000
3,562141206,1,0.428571,109.000000,17.428571
4,319550044,71,25.428571,118.569443,25.857143


In [20]:
testing_df.describe()

,customer_id,frequency,recency,monetary_value,T
count,4.688552e+07,4.688552e+07,4.688552e+07,4.688552e+07,4.688552e+07
mean,6.460814e+08,7.623310e+00,9.788681e+00,2.654792e+02,1.812018e+01
std,4.155291e+08,4.463335e+01,9.529886e+00,8.245251e+03,7.579343e+00
min,3.840000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.953372e+08,1.000000e+00,0.000000e+00,4.925000e+01,1.257143e+01
50%,5.672284e+08,3.000000e+00,7.428571e+00,1.159091e+02,2.100000e+01
75%,1.115404e+09,9.000000e+00,1.942857e+01,1.914444e+02,2.457143e+01
max,1.193621e+09,2.653700e+05,2.614286e+01,9.999990e+05,2.614286e+01


In [21]:
testing_df.frequency.quantile([.25, .5, .75, .85, .95, .99])

0.25     1.0
0.50     3.0
0.75     9.0
0.85    15.0
0.95    28.0
0.99    55.0
Name: frequency, dtype: float64

In [22]:
testing_df_filter = testing_df[testing_df.frequency < testing_df.frequency.quantile(.99)]

In [23]:
testing_df_filter.frequency.quantile([.25, .5, .75, .85, .95, .99])

0.25     1.0
0.50     3.0
0.75     9.0
0.85    14.0
0.95    26.0
0.99    42.0
Name: frequency, dtype: float64

In [24]:
testing_df_filter.monetary_value.quantile([.25, .5, .75, .85, .95, .99])

0.25     49.000000
0.50    116.107140
0.75    192.791672
0.85    256.750000
0.95    399.000000
0.99    559.000000
Name: monetary_value, dtype: float64

### as one

In [25]:
post_df = ( testing_df_filter >> rename(frequency_holdout = X.frequency, id=X.customer_id, money_holdout=X.monetary_value) 
          >> select(X.frequency_holdout, X.id, X.money_holdout))
post_df.head(1)

,frequency_holdout,id,money_holdout
0,50,378479488,95.333336


In [26]:
pre_df = ( training_df_filter >> rename(frequency_cal = 'frequency', recency_cal = 'recency', T_cal='T', id='customer_id') ) 
pre_df.head(1)

,id,frequency_cal,recency_cal,monetary_value,T_cal
0,445827048,41,22.285714,135.857147,25.714286


In [27]:
pre_post_df = pre_df >> left_join(post_df, by='id')

In [28]:
pre_post_df.head(1)

,id,frequency_cal,recency_cal,monetary_value,T_cal,frequency_holdout,money_holdout
0,445827048,41,22.285714,135.857147,25.714286,NaN,NaN


In [29]:
pre_post_df.frequency_holdout.fillna(0.0, inplace=True)

In [30]:
pre_post_df['duration_holdout'] = test_duration

In [31]:
pre_post_df.money_holdout.fillna(0.0, inplace=True)

In [32]:
pre_post_df.head(5)

,id,frequency_cal,recency_cal,monetary_value,T_cal,frequency_holdout,money_holdout,duration_holdout
0,445827048,41,22.285714,135.857147,25.714286,0.0,0.000000,26.142857
1,531467720,40,22.000000,149.414627,24.000000,54.0,91.945457,26.142857
2,243358458,9,21.285714,887.099976,22.000000,9.0,500.600006,26.142857
3,493553170,19,23.857143,197.850006,24.714286,21.0,181.727280,26.142857
4,428777360,14,12.714286,46.599998,15.428571,0.0,0.000000,26.142857


In [33]:
len(pre_post_df)

58021012

## feature n

In [34]:
pre_post_df['y'] = np.log((pre_post_df['frequency_holdout'] + 1) * pre_post_df['money_holdout'] + 1)

In [35]:
X = pre_post_df[['frequency_cal', 'recency_cal', 'monetary_value', "T_cal"]]
X.head(5)

,frequency_cal,recency_cal,monetary_value,T_cal
0,41,22.285714,135.857147,25.714286
1,40,22.000000,149.414627,24.000000
2,9,21.285714,887.099976,22.000000
3,19,23.857143,197.850006,24.714286
4,14,12.714286,46.599998,15.428571


In [36]:
y=pre_post_df['y']
y.describe()

count    5.802101e+07
mean     3.075114e+00
std      3.375993e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      6.614726e+00
max      1.774734e+01
Name: y, dtype: float64

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [38]:
def pipe(X, y, freq_scaler=None, recency_scaler=None, t_scaler=None, money_scaler=None, training=True):
    
    from sklearn.preprocessing import StandardScaler
    
    if training:
        freq_scaler = StandardScaler()
        X['sfrequency'] = freq_scaler.fit_transform(X[['frequency_cal']])
        
        recency_scaler = StandardScaler()
        X['srecency'] = recency_scaler.fit_transform(X[['recency_cal']])

        t_scaler = StandardScaler()
        X['T'] = t_scaler.fit_transform(X[['T_cal']])

        money_scaler = StandardScaler()
        X['smonetary_value'] = money_scaler.fit_transform(X[['monetary_value']])  
    else:
        X['sfrequency'] = freq_scaler.transform(X[['frequency_cal']])
        X['srecency'] = recency_scaler.transform(X[['recency_cal']])
        X['T'] = t_scaler.transform(X[['T_cal']])
        X['smonetary_value'] = money_scaler.transform(X[['monetary_value']])
        
        
    X['sqrt_freq'] = np.sqrt(X['frequency_cal']+1)
    X['log_freq'] = np.log(X['frequency_cal']+1)
    X['total'] = np.log((X['frequency_cal']+1) * X['monetary_value']+1)
    
    return X, y, freq_scaler, recency_scaler, t_scaler, money_scaler

In [39]:
X_train_p, y_train_p, freq_scaler, recency_scaler, t_scaler, money_scaler = pipe(X_train, y_train)

In [40]:
X_train_p = X_train_p[['sfrequency', 'T', 'smonetary_value', 'srecency', 'sqrt_freq', 'log_freq', 'total']]
X_train_p.head(5)

,sfrequency,T,smonetary_value,srecency,sqrt_freq,log_freq,total
30840800,-0.540411,-0.310374,-0.025485,0.387569,1.414214,0.693147,4.262680
39097024,-0.410567,1.081122,-0.025485,-0.893268,1.732051,1.098612,4.663439
16143208,1.017722,1.160636,-0.012322,1.540323,3.741657,2.639057,7.603399
25796527,-0.410567,0.464888,0.008030,0.419590,1.732051,1.098612,6.837333
42422898,-0.540411,0.564280,-0.023051,-0.605080,1.414214,0.693147,4.709530


In [41]:
X_test_p, y_test_p, *arg = pipe(X_test, y_test, freq_scaler, recency_scaler, t_scaler, money_scaler, False)

In [42]:
X_test_p = X_test_p[['sfrequency', 'T', 'smonetary_value', 'srecency', 'sqrt_freq', 'log_freq', 'total']]
X_test_p.head(5)

,sfrequency,T,smonetary_value,srecency,sqrt_freq,log_freq,total
44280220,1.666944,1.101000,-0.015312,1.652396,4.358899,2.944439,7.720462
39550098,0.108811,0.524523,-0.012896,1.412239,2.645751,1.945910,6.877296
22519280,0.887877,1.140757,-0.017058,1.492291,3.605551,2.564949,7.212295
13937755,-0.670255,0.126953,0.000077,-0.893268,1.000000,0.000000,5.505332
36388177,-0.021033,-1.344057,-0.025587,-0.028703,2.449490,1.791759,5.327876


## train

In [43]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train_p, y_train_p)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [44]:
reg.coef_

array([-1.49297231, -0.74293903, -0.06501071,  1.75757866,  2.22103053,
       -1.07603965,  0.46013828])

In [45]:
from sklearn.metrics import mean_absolute_error
y_pred=reg.predict(X_train_p)
mean_absolute_error(y_train_p, y_pred)

1.9840700742370003

## evaluation

In [46]:
y_pred=reg.predict(X_test_p)
mean_absolute_error(y_test_p, y_pred)

1.9843516433078083